In [1]:
# importing selenium
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings .filterwarnings("ignore")
import time
from selenium.common.exceptions import NoSuchElementException
import time
import requests
import re

## Q1
. Write a python program which searches all the product under a particular product from www.amazon.in.  The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.
 
## Q2.
In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [53]:
#connecting to web driver
driver=webdriver.Chrome('chromedriver.exe')
# or driver=webdriver.Chrome(r'"C:\Users\hp\Dropbox\PC\Documents\chromedriver.exe"')
url=' https://www.amazon.in/'
driver.get(url)

In [54]:
# finding web element for search using id
Product=driver.find_element_by_id('twotabsearchtextbox')
Product
# writing on search bar
Product.send_keys(input())

# click button using xpath
search_btn=driver.find_element_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input")
search_btn.click()

guitar


In [55]:

#for extracting data from multiple pages weare using for loop along with exception handling
start=0
end=3

URL=[]
for page in range(start,end):
    try:
        for i in driver.find_elements_by_xpath('//a[@class="a-link-normal s-link-style a-text-normal"]'):   
            URL.append(i.get_attribute("href"))                      # gathering url from href attribute
    except:
        URL.append("-")
# for going to next page
    time.sleep(2)
    nxt_page=driver.find_elements_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    try:
        driver.get(nxt_page[0].get_attribute('href'))
    except:
        driver.get(nxt_page[1].get_attribute('href'))
        

len(URL)

184

In [59]:
from selenium.common.exceptions import NoSuchElementException
brand = []
name = []
price = []
exchange = []
delivery=[]
availability=[]
for i in URL:
    driver.get(i)
    time.sleep(2)
    try:                        
        brnd=driver.find_element_by_xpath('//td[@class="a-span9"]/span')
        brand.append(brnd.text)                                  
    except NoSuchElementException:
        brand.append("-")
        
    try:
        nm=driver.find_element_by_xpath('//h1[@class="a-size-large a-spacing-none"]/span')            ## product name
        name.append(nm.text)
    except NoSuchElementException:
        price.append("-")
        
    try:
        prc=driver.find_element_by_xpath('//div[@class="a-section"]/span')      # price
        price.append(prc.text)
    except NoSuchElementException:
        price.append("-")
         
    try:
        exc=driver.find_element_by_xpath('//div[@class="a-section a-spacing-none icon-content"]/a')            # exchange
        exchange.append(exc.text)
    except NoSuchElementException:
        exchange.append("-")
    
    try:
        de=driver.find_element_by_xpath('//div[@class="a-section a-spacing-mini"]/b')
        delivery.append(de.text)
    except NoSuchElementException:
        delivery.append("-")
    
    try:
        ava=driver.find_element_by_xpath('//span[@class="a-size-medium a-color-success"]')
        availability.append(ava.text)
    except NoSuchElementException:
        availability.append("-")

In [61]:
print(len(brand))              #here I have checked the length of each data so that while creating a dataframe I don't face any error.
print(len(price)) 
print(len(name))
print(len(exchange))
print(len(delivery))
print(len(availability))

184
184
184
184
184
184


In [62]:
# Creating the DataFrame - 
Guitar = pd.DataFrame({'Brand Name':brand,'Name of the Product':name,'Price':price,'Return/Exchange':exchange,'Expected Delivery':delivery,'Availability':availability,'Product URL':URL})
Guitar

,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,,Pay on Delivery,"Sunday, Jan 30",,https://www.amazon.in/gp/slredirect/picassoRed...
1,VAULT,Vault EA40 Premium 41 inch Spruce-Top Cutaway ...,"₹6,990\n00",Pay on Delivery,"Friday, Jan 28",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Kadence,"Kadence Guitar Acoustica Series, Electric Acou...",,Pay on Delivery,"Sunday, Jan 30",,https://www.amazon.in/gp/slredirect/picassoRed...
3,Kadence,Kadence Frontier guitar with Online Guitar lea...,,Pay on Delivery,"Friday, Jan 28",,https://www.amazon.in/gp/slredirect/picassoRed...
4,JUAREZ,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...","₹2,190\n00",Pay on Delivery,"Friday, Jan 28",In stock.,https://www.amazon.in/Juarez-Acoustic-Guitar-C...
...,...,...,...,...,...,...,...
179,YAMAHA,Yamaha FS100C Acoustic Guitar Concert body Wit...,"₹12,831\n00",7 Days Replacement,"Sunday, Jan 30",Only 1 left in stock.,https://www.amazon.in/Yamaha-FS100C-6-Strings-...
180,MUSTANG,Pack of 2 Universal Guitar Stand by Hola! Musi...,₹999\n00,7 Days Replacement,"Monday, Jan 31",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
181,Techblaze,Techblaze 6 Pieces Guitar Keys for Acoustic Gu...,₹549\n00,Pay on Delivery,"Monday, Jan 31",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
182,TECHBLAZE,TECHBLAZE 6 Pcs Guitar Keys for Acoustic Guita...,₹589\n00,Pay on Delivery,"Monday, Jan 31",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...


In [63]:
# Saving the DataFrame as csv file for our future use.
Guitar.to_csv("Guitar Details.csv")


## Q3.
Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [91]:
#connecting to web driver
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.google.com/imghp?hl=en'
driver.get(url)

In [92]:
# finding web element for search using id
ig=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
ig
# writing on search bar
ig.send_keys(input())
time.sleep(3)
# click button using xpath
search_btn=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_btn.click()

fruits


In [93]:
for i in range(500):
    driver.execute_script("window.scrollBy(0,1000)")                       #Scrolling the windows of the search page.
    image = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []                                                               #Empty list
for img in image: 
    source = img.get_attribute('src')                                    #Source of the image's
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)  

205

In [95]:
import requests                                                    
for i in range(len(img_urls)):                                             #Here we have used range fuc. and inside that we have given len fuc.(due to this our code will run the same no. of time as the lenth of img.)
    if i>10:                                                               #We want to download 10 images.
        break
    print("Downloading {0} of {1} images".format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:\image\q3"+str(i)+".jpg","wb")   #Opeing the folder location where we want to download
    file.write(response.content)

In [96]:
# for cars
#connecting to web driver
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.google.com/imghp?hl=en'
driver.get(url)

In [97]:
# finding web element for search using id
ig=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
ig
# writing on search bar
ig.send_keys(input())
time.sleep(3)
# click button using xpath
search_btn=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_btn.click()

cars


In [99]:
for i in range(500):
    driver.execute_script("window.scrollBy(0,1000)")                       #Scrolling the windows of the search page.
    image = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []                                                               #Empty list
for img in image: 
    source = img.get_attribute('src')                                    #Source of the image's
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls) 
for i in range(len(img_urls)):                                             #Here we have used range fuc. and inside that we have given len fuc.(due to this our code will run the same no. of time as the lenth of img.)
    if i>10:                                                               #We want to download 10 images.
        break
    print("Downloading {0} of {1} images".format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:\image\q3"+str(i)+".jpg","wb")   #Opeing the folder location where we want to download
    file.write(response.content)

In [102]:
# for machine learning
#connecting to web driver
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.google.com/imghp?hl=en'
driver.get(url)
# finding web element for search using id
ig=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
ig
# writing on search bar
ig.send_keys(input())
time.sleep(3)
# click button using xpath
search_btn=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_btn.click()

machine learning


In [103]:
for i in range(500):
    driver.execute_script("window.scrollBy(0,1000)")                       #Scrolling the windows of the search page.
    image = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []                                                               #Empty list
for img in image: 
    source = img.get_attribute('src')                                    #Source of the image's
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls) 
for i in range(len(img_urls)):                                             #Here we have used range fuc. and inside that we have given len fuc.(due to this our code will run the same no. of time as the lenth of img.)
    if i>10:                                                               #We want to download 10 images.
        break
    print("Downloading {0} of {1} images".format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:\image\q3"+str(i)+".jpg","wb")   #Opeing the folder location where we want to download
    file.write(response.content)

In [104]:
# for guitar
#connecting to web driver
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.google.com/imghp?hl=en'
driver.get(url)
# finding web element for search using id
ig=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
ig
# writing on search bar
ig.send_keys(input())
time.sleep(3)
# click button using xpath
search_btn=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_btn.click()

Guitar


In [105]:
for i in range(500):
    driver.execute_script("window.scrollBy(0,1000)")                       #Scrolling the windows of the search page.
    image = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []                                                               #Empty list
for img in image: 
    source = img.get_attribute('src')                                    #Source of the image's
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls) 
for i in range(len(img_urls)):                                             #Here we have used range fuc. and inside that we have given len fuc.(due to this our code will run the same no. of time as the lenth of img.)
    if i>10:                                                               #We want to download 10 images.
        break
    print("Downloading {0} of {1} images".format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:\image\q3"+str(i)+".jpg","wb")   #Opeing the folder location where we want to download
    file.write(response.content)

In [106]:
# for cakes
#connecting to web driver
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.google.com/imghp?hl=en'
driver.get(url)
# finding web element for search using id
ig=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
ig
# writing on search bar
ig.send_keys(input())
time.sleep(3)
# click button using xpath
search_btn=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_btn.click()

Cakes


In [107]:
for i in range(500):
    driver.execute_script("window.scrollBy(0,1000)")                       #Scrolling the windows of the search page.
    image = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []                                                               #Empty list
for img in image: 
    source = img.get_attribute('src')                                    #Source of the image's
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls) 
for i in range(len(img_urls)):                                             #Here we have used range fuc. and inside that we have given len fuc.(due to this our code will run the same no. of time as the lenth of img.)
    if i>10:                                                               #We want to download 10 images.
        break
    print("Downloading {0} of {1} images".format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:\image\q3"+str(i)+".jpg","wb")   #Opeing the folder location where we want to download
    file.write(response.content)

## Q4.
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [29]:
#connecting to web driver
driver=webdriver.Chrome('chromedriver.exe')
url='https://www.flipkart.com/'
driver.get(url)

In [30]:
# finding web element for search using id
smartphone=driver.find_element_by_class_name("_3704LK")
smartphone
# writing on search bar
smartphone.send_keys("realme")

# click button using xpath
search_btn=driver.find_element_by_class_name("L0Z3Pu")
search_btn.click()

In [34]:

#for extracting data from multiple pages weare using for loop along with exception handling

url=[]
try:
    for i in driver.find_elements_by_xpath('//div[@class="_2kHMtA"]/a'): 
        url.append(i.get_attribute("href"))
           
except:
     url.append("-")
        

len(url)

40

In [35]:

from selenium.common.exceptions import NoSuchElementException
brand = []
name = []
colour = []
ram = []
rom=[]
pcam=[]
scam=[]
dsize=[]
battery=[]
price=[]
for i in url:
    driver.get(i)
    time.sleep(2)
    but=driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')
    but.click()
    try:                        
        b=driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # for brand
        brand.append(b.text.split()[0]) 
    except NoSuchElementException:
        brand.append("-")
        
    try:
        n=driver.find_element_by_xpath('//span[@class="B_NuCI"]')            ## product name
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")
        
    try:
        c=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li')      # for colour
        colour.append(c.text)
    except NoSuchElementException:
        colour.append("-")
         
    try:
        r=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li')            #for ram 
        ram.append(r.text)
    except NoSuchElementException:
        ram.append("-")
    
    try:
        ro=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        rom.append(ro.text)
    except NoSuchElementException:
        rom.append("-")
    
    try:
        pc=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li')
        pcam.append(pc.text)
    except NoSuchElementException:
        pcam.append("-")
    try:
        s=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[5]/td[2]/ul/li')
        scam.append(s.text)
    except NoSuchElementException:
        scam.append("-")
    try:
        d=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        dsize.append(d.text)
    except NoSuchElementException:
        dsize.append("-")
    try:
        ba=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[9]/table/tbody/tr/td[2]/ul/li')
        battery.append(ba.text)
    except NoSuchElementException:
        battery.append("-")
    try:
        pr=driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        price.append(pr.text)
    except NoSuchElementException:
        [price].append("-")

In [36]:
print(len(brand))                     #Here I have printed the lenth of each of the data extracted so that we don't face any problem in creating a DataFrame.
print(len(name))
print(len(colour))
print(len(ram))
print(len(rom))
print(len(pcam))
print(len(scam))
print(len(dsize))
print(len(battery))
print(len(price))

40
40
40
40
40
40
40
40
40
40


In [40]:
# Creating the DataFrame - 
phone_detail = pd.DataFrame({'Brand Name':brand,'Name of the Product':name,'Colour':colour,'Ram':ram,'Rom':rom,'Primary Cam':pcam,'Secondary Cam':scam,'Display size':dsize,'Battery':battery,'Price':price})
phone_detail.head()

,Brand Name,Name of the Product,Colour,Ram,Rom,Primary Cam,Secondary Cam,Display size,Battery,Price
0,realme,"realme Narzo 50i (Mint Green, 32 GB) (2 GB RAM)",Mint Green,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹7,499"
1,realme,"realme Narzo 50i (Carbon Black, 32 GB) (2 GB ...",Carbon Black,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹7,499"
2,realme,"realme Narzo 50i (Mint Green, 64 GB) (4 GB RAM)",Mint Green,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹8,999"
3,realme,"realme Narzo 50i (Carbon Black, 64 GB) (4 GB ...",Carbon Black,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹8,999"
4,realme,"realme Narzo 50A (Oxygen Blue, 64 GB) (4 GB RAM)",Oxygen Blue,4 GB,64 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),6000 mAh,"₹11,499"


In [21]:
# Saving the DataFrame as csv file for our future use.
phone_detail.to_csv("Smartphone Details.csv")


### Q5
 Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.


In [2]:
import re
driver = webdriver.Chrome("chromedriver.exe")
url="https://www.google.co.in/maps"
driver.get(url)
time.sleep(10)

In [3]:
# finding web element for search using id
gmap=driver.find_element_by_class_name("tactile-searchbox-input")
gmap
# writing on search bar
gmap.send_keys("tokyo")

# click button using xpath
search_btn=driver.find_element_by_class_name("nhb85d-BIqFsb")
search_btn.click()

In [4]:
try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, \nLongitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/Tokyo,+Japan/@35.4884661,137.5260993,7z/data=!3m1!4b1!4m5!3m4!1s0x605d1b87f02e57e7:0x2e01618b22571b89!8m2!3d35.6761919!4d139.6503106
Latitude = 35.4884661, 
Longitude = 137.5260993


## Q6
Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in

In [16]:
# connecting web driver
driver = webdriver.Chrome("chromedriver.exe")              
url = 'https://trak.in/' # website url
driver.get(url)
time.sleep(2)  
fund = driver.find_element_by_xpath('/html/body/div[1]/header/div[2]/div/div/div/div/nav/ul/li[9]/a')
fund.click()

In [17]:
f= driver.find_element_by_xpath('/html/body/div[1]/header/div[2]/div/div/div/div/nav/ul/li[9]/a')
f.click()

In [31]:
serial = []                   #Empty List for the data we want to extract
date = []
startup = []
industry = []
sub = []
city = []
investment = []
typ = []
amount = []

s = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[1]')  #Extracting the table data by the xpath.
d = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[2]')
st = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[3]')
ind = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[4]')
su = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[5]')
c = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[6]')
inv = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[7]')
t = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[8]')
a = driver.find_elements_by_xpath('//*[@id="tablepress-54"]/tbody/tr/td[9]')


for i in s:
    serial.append(i.text)    #Appending the text data.
for j in d:
    date.append(j.text)
for k in st:
    startup.append(k.text)
for l in ind:
    industry.append(l.text)
for m in su:
    sub.append(m.text)
for n in c:
    city.append(n.text)
for o in inv:
    investment.append(o.text)
for p in t:
    typ.append(p.text)
for q in a:
    amount.append(q.text)
    
print(len(serial))     # Printing the length of each of the Data Extracted so that while creating the DataFrame we don't face any problem.
print(len(date))
print(len(startup))
print(len(industry))
print(len(sub))
print(len(city))
print(len(investment))
print(len(typ))
print(len(amount))

8
8
8
8
8
8
8
8
8


In [37]:
# creating dataframe
january=pd.DataFrame({'serial':serial,'Date':date,'Startup':startup,'Industry':industry,'Sub-Vertical':sub,'City':city,'Investment':investment,'Type':typ,'Amount':amount})


### Funding Deal (January-21)

In [40]:
january

,serial,Date,Startup,Industry,Sub-Vertical,City,Investment,Type,Amount
0,1,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,2,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,3,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,5,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,6,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,7,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,8,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"


In [41]:
serial = []                   #Empty List for the data we want to extract
date = []
startup = []
industry = []
sub = []
city = []
investment = []
typ = []
amount = []

s = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[1]')  #Extracting the table data by the xpath.
d = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[2]')
st = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[3]')
ind = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[4]')
su = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[5]')
c = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[6]')
inv = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[7]')
t = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[8]')
a = driver.find_elements_by_xpath('//*[@id="tablepress-55"]/tbody/tr/td[9]')


for i in s:
    serial.append(i.text)    #Appending the text data.
for j in d:
    date.append(j.text)
for k in st:
    startup.append(k.text)
for l in ind:
    industry.append(l.text)
for m in su:
    sub.append(m.text)
for n in c:
    city.append(n.text)
for o in inv:
    investment.append(o.text)
for p in t:
    typ.append(p.text)
for q in a:
    amount.append(q.text)
    
print(len(serial))     # Printing the length of each of the Data Extracted so that while creating the DataFrame we don't face any problem.
print(len(date))
print(len(startup))
print(len(industry))
print(len(sub))
print(len(city))
print(len(investment))
print(len(typ))
print(len(amount))

9
9
9
9
9
9
9
9
9


In [43]:
# creating dataframe
February=pd.DataFrame({'serial':serial,'Date':date,'Startup':startup,'Industry':industry,'Sub-Vertical':sub,'City':city,'Investment':investment,'Type':typ,'Amount':amount})


### Funding Deal (February-21)

In [44]:
February

,serial,Date,Startup,Industry,Sub-Vertical,City,Investment,Type,Amount
0,1,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
1,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
2,3,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"
3,4,17/02/2021,Zolve,FinTech,Global Neobank Venture,Mumbai,Accel Partners and Lightspeed Venture Partners,Seed,"1,50,00,000"
4,5,15/02/2021,KreditBee,Finance,Digital lending platform,Bengaluru,Azim Premji’s PremjiInvest and South Korea’s M...,Series C,"75,000,000"
5,6,12/02/2021,Pepperfry,E-commerce,Multi-brand furniture brand,Mumbai,InnoVen Capital,Debt Financing,"4,773,958"
6,7,12/02/2021,Grofers,E-Commerce,Online supermarket,Gurgaon,SoftBank Vision Fund (SVF),Unspecified,"55,000,000"
7,8,09/02/2021,Nothing,Technology,Consumer Technology Venture,London,GV,Series A,"15,000,000"
8,9,09/02/2021,SplashLearn,EdTech,Game-based learning programme,Gurgaon,Owl Ventures,Series C,"18,000,000"


In [45]:
serial = []                   #Empty List for the data we want to extract
date = []
startup = []
industry = []
sub = []
city = []
investment = []
typ = []
amount = []

s = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[1]')  #Extracting the table data by the xpath.
d = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[2]')
st = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[3]')
ind = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[4]')
su = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[5]')
c = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[6]')
inv = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[7]')
t = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[8]')
a = driver.find_elements_by_xpath('//*[@id="tablepress-56"]/tbody/tr/td[9]')


for i in s:
    serial.append(i.text)    #Appending the text data.
for j in d:
    date.append(j.text)
for k in st:
    startup.append(k.text)
for l in ind:
    industry.append(l.text)
for m in su:
    sub.append(m.text)
for n in c:
    city.append(n.text)
for o in inv:
    investment.append(o.text)
for p in t:
    typ.append(p.text)
for q in a:
    amount.append(q.text)
    
print(len(serial))     # Printing the length of each of the Data Extracted so that while creating the DataFrame we don't face any problem.
print(len(date))
print(len(startup))
print(len(industry))
print(len(sub))
print(len(city))
print(len(investment))
print(len(typ))
print(len(amount))

7
7
7
7
7
7
7
7
7


In [46]:
# creating dataframe
march=pd.DataFrame({'serial':serial,'Date':date,'Startup':startup,'Industry':industry,'Sub-Vertical':sub,'City':city,'Investment':investment,'Type':typ,'Amount':amount})


### Funding Deal (March-21)

In [47]:
march

,serial,Date,Startup,Industry,Sub-Vertical,City,Investment,Type,Amount
0,1,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,2,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,3,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,4,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,5,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
5,6,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
6,7,26/03/2021,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"


## Q7
Write a program to scrap all the available details of best gaming laptops from digit.in

In [48]:
# connecting web driver
driver = webdriver.Chrome("chromedriver.exe")              
url = 'https://www.digit.in/' # website url
driver.get(url)
time.sleep(2)  
search = driver.find_element_by_xpath('/html/body/div[4]/div/div[2]/div[2]/div[4]/ul/li[9]/a') 
search.click()                # clicking on best gaming laptop

In [96]:
name = []                   #Empty List for the data we want to extract
os = []
disp = []
pro = []
mem = []




n = driver.find_elements_by_tag_name('h3')   #Extracting the table data by the xpath.
o = driver.find_elements_by_xpath('//div[@class="product-detail"]/div/ul/li[1]/div')
d = driver.find_elements_by_xpath('//div[@class="product-detail"]/div/ul/li[2]/div')
p = driver.find_elements_by_xpath('//div[@class="product-detail"]/div/ul/li[4]/div')
mm = driver.find_elements_by_xpath('//div[@class="product-detail"]/div/ul/li[5]/div')




for i in n:
    name.append(i.text)    #Appending the text data.
for j in o:
    os.append(j.text.split('\n')[1])
for k in d:
    disp.append(k.text.split('\n')[1])
for l in p:
    pro.append(l.text.split('\n')[1])
for m in mm:
    mem.append(m.text.split('\n')[1])
prices=driver.find_elements_by_xpath("//td[@class='smprice']")
for n in pp:
    if n.text is NoSuchElementException:
        pr.append('-')
    else:
        pr.append(n.text)
       
   


    
print(len(name))     # Printing the length of each of the Data Extracted so that while creating the DataFrame we don't face any problem.
print(len(os))
print(len(disp))
print(len(pro))
print(len(mem))




10
10
10
10
10


In [99]:
# creating dataframe
Laptop=pd.DataFrame({'Name':name,'OS':os,'Display':disp,'Processor':pro,'Memory':mem})
Laptop

,Name,OS,Display,Processor,Memory
0,ACER NITRO 5,WINDOWS 10,"15.6"" (1920 X 1080)",AMD RYZEN 9 OCTA CORE | 2.4 GHZ,1 TB HDD/16 GBGB DDR4
1,MSI STEALTH 15M,WINDOWS 10,"15.6"" (1920 X 1080)",INTEL CORE I7 11TH GEN - 11375H | NA,1 TB SSD/16 GBGB DDR4
2,ASUS ROG STRIX SCAR 15,WINDOWS 10,"15.6"" (2560 X 1440)",AMD RYZEN 9 OCTA CORE - 5900HX | 3.3 GHZ,2 TB SSD/32 GBGB DDR4
3,ALIENWARE AREA 51M R2,WINDOWS 10 HOME,"17.3"" (1920 X 1080)",10TH GEN INTEL® CORE™ I7-10700 | 2.90 GHZ,1 TB SSD/16 GBGB DDR4
4,ALIENWARE M15 R3,WINDOWS 10 HOME,"15.6"" (3840 X 2160)",10TH GEN INTEL® CORE™ I9-10980HK | NA,1 TB SSD/16 GBGB DDR4
5,ASUS ROG STRIX SCAR 15,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN™ 9 5900HX | 3.3 GHZ,1 TB SSD/16 GBGB DDR4
6,ASUS ZEPHYRUS G14,WINDOWS 10 HOME,"14"" (1920 X 1080)",AMD 3RD GEN RYZEN 9 | 3.3 GHZ,1 TB SSD/16 GBGB DDR4
7,LENOVO LEGION 5I,WINDOWS 10 PRO,"15.6"" (1920 X 1080)",10TH GEN INTEL® CORE™ I5-10300H | 2.50 GHZ,1 TB SSD/16 GBGB DDR4
8,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10,"15.6"" (3840 X 1100)",INTEL CORE I7 10TH GEN 10875H | NA,512 GB SSD/4 GBGB DDR4
9,ACER ASPIRE 7 GAMING LAPTOP,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD RYZEN™ 5-5500U HEXA-CORE | NA,512 GB SSD/8 GBGB DDR4


## Q8.
 Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.


In [106]:
driver = webdriver.Chrome("chromedriver.exe")
url = "https://www.forbes.com/billionaires/"
driver.get(url)

In [109]:
rank=[]
name = []                   #Empty List for the data we want to extract
net = []
age = []
city = []
source = []
ind=[]

rr = driver.find_elements_by_xpath('//div[@class="rank"]') #Extracting the table data by the xpath.
nn = driver.find_elements_by_xpath('//div[@class="personName"]')
ne = driver.find_elements_by_xpath('//div[@class="netWorth"]')
aa = driver.find_elements_by_xpath('//div[@class="age"]')
cc = driver.find_elements_by_xpath('//div[@class="countryOfCitizenship"]')
ss = driver.find_elements_by_xpath('//div[@class="source-column"]')
ii = driver.find_elements_by_xpath('//div[@class="category"]')


for i in rr:
    rank.append(i.text)    #Appending the text data.
for j in nn:
    name.append(j.text)
for k in ne:
    net.append(k.text)
for l in aa:
    age.append(l.text)
for m in cc:
    city.append(m.text)
for n in ss:
    source.append(n.text)
for o in ii:
    ind.append(o.text)

    
print(len(rank))     # Printing the length of each of the Data Extracted so that while creating the DataFrame we don't face any problem.
print(len(name))
print(len(net))
print(len(age))
print(len(city))
print(len(source))
print(len(ind))




200
200
200
200
200
200
200


In [110]:
# creating dataframe
Billionaires=pd.DataFrame({'Rank':rank,'Name':name,'Net- Worth':net,'Age':age,'Citizenship':city,'Source':source,'Industry':ind})
Billionaires

,Rank,Name,Net- Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,195.,Harry Triguboff,$11.2 B,88,Australia,real estate,Real Estate
196,197.,Leonid Fedun & family,$11.1 B,65,Russia,oil,Energy
197,197.,Eyal Ofer,$11.1 B,70,Israel,"real estate, shipping",Diversified
198,197.,Evan Spiegel,$11.1 B,30,United States,Snapchat,Technology


## Q9
 Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video.

In [126]:
url = 'https://www.youtube.com/' #opening the website
driver.get(url)

search = driver.find_element_by_xpath("/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input") # locating search bar
time.sleep(2)
search.send_keys(input())    # Asking the User to input which youtube video he/she wants's to enter in search bar.

button = driver.find_element_by_id('search-icon-legacy')        # locating search button
button.click()                                                  # clicking search button

time.sleep(5)

url=[]                                                         # Extracting the url of very First youtube video 
fir=driver.find_elements_by_xpath('//h3[@class="title-and-badge style-scope ytd-video-renderer"]/a')
for i in fir[0:1]:
    url.append(i.get_attribute('href'))             # href for 1st video
url

time.sleep(5)

for i in url:                                                  
    driver.get(i)                                              

numb


In [127]:
for j in range(1000):
    driver.execute_script("window.scrollBy(0,500)")           # Scrolling that particular Youtube video to the last so, that all the comment's,likes and uplikes get's visible. 
time.sleep(5)
com = []                                                      # Giving empty list for all the data what all I want to extract 
upvote=[]
time=[]
comm =driver.find_elements_by_xpath('//*[@id="content-text"]')  # Giving the xpath of the desired element.
vote=driver.find_elements_by_id('vote-count-middle')
ti=driver.find_elements_by_id('header-author')
for k in comm:
    com.append(k.text.replace('\n',""))           # Appending the text data and replacing the unproductive data by "\n"
for l in vote:
    upvote.append(l.text)
for m in ti:
    time.append(m.text.split('\n')[-1])
print(len(com))                                   # Printing the length
print(len(upvote))
print(len(time))

80
80
80


In [128]:
# creating dataframe
youtube=pd.DataFrame({'Comment':com,'Upvote':upvote,'Time of comment':time})
youtube

,Comment,Upvote,Time of comment
0,In the Next 10 years I’ll still enjoy this son...,916,1 year ago
1,Linkin Park songs have always been part of the...,795,9 days ago
2,"Siempre fue un grande, descansa en paz Chester...",9,17 hours ago (edited)
3,Who is listening to this amazing music in 2022 ?,5.1K,3 weeks ago
4,"Essa música me trás uma nostalgia incrível, nã...",10,12 hours ago
...,...,...,...
75,"If you're reading this, you're a legendR.I.P C...",47,3 weeks ago
76,A vibe dessa música,2,1 month ago
77,My parents thinks that I’m listening to this s...,1,2 days ago
78,LP is the only band whose every song is relata...,335,8 months ago


## Q10
Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description.

In [17]:
driver = webdriver.Chrome("chromedriver.exe")
url = 'https://www.hostelworld.com/'  # Opeing the url
driver.get(url)
time.sleep(2)
london=driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[7]/section/div/div[1]/a') # Clicking on london location
london.click()
time.sleep(5)
search = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/section[1]/div/div/form/div/div[2]/button') # Clicking on search button to search all the hostel's in London location
search.click()

In [18]:
name = []                        # Empty List
distance=[]
ratings=[]
reviews=[]
treview=[]
private=[]
dorms=[]
facilities=[]
hurl=[]

while(True):   
    nam=driver.find_elements_by_xpath('//h2[@class="title title-6"]')                # Extracting the Desired Element's by xpath
    dis=driver.find_elements_by_xpath('//div[@class="title-row"]/div/a/span[1]')
    rat=driver.find_elements_by_xpath('//div[@class="rating rating-summary-container big"]')
    rev=driver.find_elements_by_xpath('//div[@class="rating rating-summary-container big"]')
    tre=driver.find_elements_by_xpath('//div[@class="rating rating-summary-container big"]')
    pri=driver.find_elements_by_xpath('//div[@class="price-col"]')
    dor=driver.find_elements_by_xpath('//div[@class="price-col"]')
    faci=driver.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
    urls=driver.find_elements_by_xpath("//h2[@class='title title-6']/a")

    for i in nam:
        name.append(i.text)                    # Appending the Text Data
    for j in dis:
        distance.append(j.text)
    for k in rat:
        ratings.append(k.text.split('\n')[0])
    for l in rev:
        reviews.append(l.text.split('\n')[1])
    for m in tre:
            treview.append(m.text.split()[2])
    for o in pri:
        private.append(o.text.replace('\n',''))
    for p in dor:
        dorms.append(p.text.replace('\n',''))
    for q in faci:
        facilities.append(q.text)
    for url in urls:
        hurl.append(url.get_attribute('href'))
        
    for n in reviews:                            # As there were some data in 'OverAll Reviews' column some of the data were missing, so replacing those data by "none"
        if "Rating"in n:
            reviews[reviews.index(n)] = "None"
        elif "Reviews"in n:
            reviews[reviews.index(n)] = "None"
       
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')  #Clicking on next button

        next_button.click()

    except:
             break
print(len(name))                                # Printing the length of the  Extracted Data
print(len(distance))
print(len(ratings))
print(len(reviews))
print(len(treview))
print(len(private))
print(len(dorms))
print(len(facilities))
print(len(hurl))

77
77
77
77
77
154
154
77
77


In [19]:
p=private[0:200:2]
pp=dorms[1:200:2]
print(len(p))
print(len(pp))

77
77


In [20]:
# Extracting the Description of each hotels by individually going to each url of each hosel as we have already extracted the url of each Hostel's

description = []
for i in hurl:    
    try:
        driver.get(i)
        try:
            see=driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div/div/a')
            see.click()
        except:
            pass   
        time.sleep(1)
        des=driver.find_element_by_xpath('//div[@class="description-container"]')
        description.append(des.text)
    
    except NoSuchElementException:
        description.append("No Description Availabe")

In [21]:
len(description)

77

In [23]:
# Now Creating DataFrame:
hostel = pd.DataFrame({'Hostel Name':name,'Distance From City Centre':distance,'Total Reviews':treview,'Overall Reviews':reviews,'Privates From Price':p,'Dorms From Price':pp,'Facilities':facilities,'Property Description':description})
hostel

,Hostel Name,Distance From City Centre,Total Reviews,Overall Reviews,Privates From Price,Dorms From Price,Facilities,Property Description
0,Mornington Camden,Hostel - 4.1km from city centre,44,Fabulous,No Privates Available,Dorms FromRs3277,Free WiFi,Property Description\nMornington Camden Hostel...
1,Smart Russell Square Hostel,Hostel - 2.6km from city centre,9567,Good,No Privates Available,Dorms FromRs1214,Free WiFi\nFollows Covid-19 sanitation guidance,Hostelworld says\nMany Hostelworld guests who ...
2,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,2740,Fabulous,Privates FromRs5470,Dorms FromRs1733,Free WiFi\nFollows Covid-19 sanitation guidance,Property Description\nLocated in the middle of...
3,Selina Camden,Hostel - 5.5km from city centre,15,Superb,Privates FromRs12901,Dorms FromRs4173.85 Rs3548,Free WiFi,Property Description\nAmong underground music ...
4,Queen Elizabeth Chelsea,Hostel - 5.7km from city centre,Good,Very Good,Privates FromRs4112,Dorms FromRs1821,Free WiFi,Property Description\nPLEASE NOTE WE ONLY ACCE...
...,...,...,...,...,...,...,...,...
72,Green Rooms,Hotel - 10.8km from city centre,76,Superb,Privates FromRs7844,Dorms FromRs2477,Free WiFi,No Description Availabe
73,23 Matheson Road,Bed and Breakfast - 6.2km from city centre,6,None,Privates FromRs5025,No Dorms Available,,No Description Availabe
74,Park Hotel Essex,Hotel - 24.1km from city centre,109,Fabulous,Privates FromRs11636,No Dorms Available,Free Breakfast\nFollows Covid-19 sanitation gu...,No Description Availabe
75,Cranbrook Hotel,Hotel - 14.8km from city centre,58,None,Privates FromRs4148,No Dorms Available,Free Breakfast\nFollows Covid-19 sanitation gu...,No Description Availabe


In [24]:
# Saving the DataFrame as csv file
hostel. to_csv("Hostel_details.csv")

In [25]:
# Closing the Driver
driver.close()